In [1]:
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2)
# Convert to numpy arrays before splitting
X_np = X.astype('float32')
y_np = y.reshape(-1, 1).astype('float32')


# Split
x_train_np, x_test_np, y_train_np, y_test_np = train_test_split(X_np, y_np, test_size=0.2)

# Convert back to TensorFlow tensors after splitting
x_train = tf.convert_to_tensor(x_train_np, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test_np, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train_np, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test_np, dtype=tf.float32)


# Define model
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(64, activation='relu')
        self.d2 = tf.keras.layers.Dense(32, activation='relu')
        self.out = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        return self.out(x)

# Instantiate model
model = MLP()
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Training loop (batch + manual)
batch_size = 32
for epoch in range(5):
    for i in range(0, len(x_train), batch_size):
        x_batch = x_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        with tf.GradientTape() as tape:
            logits = model(x_batch)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    print(f"Epoch {epoch+1} Loss: {loss.numpy():.4f}")

# Evaluate
preds = tf.round(model(x_test))
accuracy = tf.reduce_mean(tf.cast(tf.equal(preds, y_test), tf.float32))
print("Test Accuracy:", accuracy.numpy())

Epoch 1 Loss: 0.4760
Epoch 2 Loss: 0.3133
Epoch 3 Loss: 0.2161
Epoch 4 Loss: 0.1642
Epoch 5 Loss: 0.1371
Test Accuracy: 0.9
